<a href="https://colab.research.google.com/github/Esantomi/NLP/blob/master/NLP_Day6_CharRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 글자 단위 RNN 언어 모델

'경마장에 말이 달리고 있다'  
경마장에  
말이  
달리고  
있다  
  
글자 단위  
경  
마  
장  
에  
말  
이  
달  
리  
고  
있  
다

In [ ]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

In [ ]:
urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")

('11-0.txt', <http.client.HTTPMessage at 0x7f922e82cad0>)

In [ ]:
f = open('11-0.txt', 'rb')
lines = []
for line in f:
  line = line.strip() # strip을 통해 \r, \n을 제거
  line = line.lower() # 소문자화 
  line = line.decode('ascii', 'ignore') # \we2\x80\x99등과 같은 바이트 열 제거
  if len(line)>0:
    lines.append(line)
f.close()

In [ ]:
lines[:5] 

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. you may copy it, give it away or re-use it under the terms',
 'of the project gutenberg license included with this ebook or online at']

In [ ]:
text = ' '.join(lines)
print('문자열의 길이 또는 총 글자의 갯수 : %d' %len(text))

문자열의 길이 또는 총 글자의 갯수 : 159484


In [ ]:
print(text[:10])

the projec


In [ ]:
# 글자 집합을 만들어 보자
char_vocab = sorted(list(set(text)))
vocab_size = len(char_vocab)
print('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 56


In [ ]:
print(char_vocab)

[' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
# 글자 집합에 인덱스를 부여하고 전부 출력하기
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}


In [ ]:
# 인덱스로부터 글자를 리턴하기
index_to_char = {}
for key, value in char_to_index.items():
  index_to_char[value] = key

In [ ]:
print(index_to_char)

{0: ' ', 1: '!', 2: '"', 3: '#', 4: '$', 5: '%', 6: "'", 7: '(', 8: ')', 9: '*', 10: ',', 11: '-', 12: '.', 13: '/', 14: '0', 15: '1', 16: '2', 17: '3', 18: '4', 19: '5', 20: '6', 21: '7', 22: '8', 23: '9', 24: ':', 25: ';', 26: '?', 27: '[', 28: ']', 29: '_', 30: 'a', 31: 'b', 32: 'c', 33: 'd', 34: 'e', 35: 'f', 36: 'g', 37: 'h', 38: 'i', 39: 'j', 40: 'k', 41: 'l', 42: 'm', 43: 'n', 44: 'o', 45: 'p', 46: 'q', 47: 'r', 48: 's', 49: 't', 50: 'u', 51: 'v', 52: 'w', 53: 'x', 54: 'y', 55: 'z'}


In [ ]:
# 훈련데이터를 구성
# apple 
# sample의 길이 4

# example) 샘플의 길이가 4라면 4개의 입력 글자 시퀀스로부터 4개의 출력 글자 시퀀스 예측. 즉 RNN의 time step은 4번
# appl -> pple
# appl (입력시퀀스, train_x), pple(예측해야하는 시퀀스, train_y)

In [ ]:
# 15만 8천의 길이를 가진 text 문자열로부터 다수의 문장 샘플들로 분리
# 분리하는 방법은 문장 샘플의 길이를 정하고, 해당 길이만큼 문자열 전부를 전부 등분하는 것!!

seq_length = 60
n_samples = int(np.floor((len(text)-1)/seq_length))  # 문자열을 60등분한다 ---> 총 샘플의 수

print('문장 샘플의 수 : {}'.format(n_samples))

문장 샘플의 수 : 2658


In [ ]:
train_x = []
train_y = []

for i in range(n_samples): # 2658번 수행
  x_sample = text[i * seq_length: (i+1)*seq_length] # 문장 샘플을 1개씩 가져온다.
  x_encoded = [char_to_index[c] for c in x_sample] # 하나의 문장 샘플에 대해서 정수 인코딩
  train_x.append(x_encoded)

  y_sample = text[i*seq_length + 1: (i+1) *seq_length + 1] # 오른쪽으로 1칸 쉬프트한다.
  y_encoded = [char_to_index[c] for c in y_sample]
  train_y.append(y_encoded)

In [ ]:
print(train_x[0])

[49, 37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30]


In [ ]:
print(train_y[0])

[37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30, 43]


In [ ]:
print(train_x[1])

[43, 33, 10, 0, 31, 54, 0, 41, 34, 52, 38, 48, 0, 32, 30, 47, 47, 44, 41, 41, 0, 49, 37, 38, 48, 0, 34, 31, 44, 44, 40, 0, 38, 48, 0, 35, 44, 47, 0, 49, 37, 34, 0, 50, 48, 34, 0, 44, 35, 0, 30, 43, 54, 44, 43, 34, 0, 30, 43, 54]


In [ ]:
print(train_y[1])

[33, 10, 0, 31, 54, 0, 41, 34, 52, 38, 48, 0, 32, 30, 47, 47, 44, 41, 41, 0, 49, 37, 38, 48, 0, 34, 31, 44, 44, 40, 0, 38, 48, 0, 35, 44, 47, 0, 49, 37, 34, 0, 50, 48, 34, 0, 44, 35, 0, 30, 43, 54, 44, 43, 34, 0, 30, 43, 54, 52]


In [ ]:
# x와 y에 대해 원-핫 인코딩 수행. 입력시퀀스에 대해 워드 임베딩 하지 않습니다 -> embedding layer사용 X
train_x = to_categorical(train_x)
train_y = to_categorical(train_y)

In [ ]:
print('train_x의 크기 (shape) : {}'.format(train_x.shape))
print('train_y의 크기 (shape) : {}'.format(train_y.shape))

train_x의 크기 (shape) : (2658, 60, 56)
train_y의 크기 (shape) : (2658, 60, 56)


- 샘플의 수가 2658, 입력시퀀스의 길이(input_length) 60, 각 벡터의 차원(input_dim) 56 의미  
- 원-핫 벡터의 차원은 글자 집합의 크기인 56

![](https://wikidocs.net/images/page/22886/rnn_image6between7.PNG)

## 모델 설계하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(None, train_x.shape[2]), return_sequences=True))  # None은 어떤 크기든 무관 (크기 미지정)
model.add(LSTM(256, return_sequences=True))  # 256은 임의 값 (128, 256, 512 ...)
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.fit(train_x, train_y, epochs=80, verbose=1)

Epoch 1/80
84/84 [==============================] - 9s 11ms/step - loss: 3.0730 - accuracy: 0.1811
Epoch 2/80
84/84 [==============================] - 1s 10ms/step - loss: 2.7692 - accuracy: 0.2379
Epoch 3/80
84/84 [==============================] - 1s 10ms/step - loss: 2.3862 - accuracy: 0.3308
Epoch 4/80
84/84 [==============================] - 1s 10ms/step - loss: 2.2341 - accuracy: 0.3668
Epoch 5/80
84/84 [==============================] - 1s 10ms/step - loss: 2.1242 - accuracy: 0.3931
Epoch 6/80
84/84 [==============================] - 1s 10ms/step - loss: 2.0373 - accuracy: 0.4144
Epoch 7/80
84/84 [==============================] - 1s 10ms/step - loss: 1.9661 - accuracy: 0.4335
Epoch 8/80
84/84 [==============================] - 1s 10ms/step - loss: 1.9022 - accuracy: 0.4513
Epoch 9/80
84/84 [==============================] - 1s 10ms/step - loss: 1.8440 - accuracy: 0.4677
Epoch 10/80
84/84 [==============================] - 1s 10ms/step - loss: 1.7926 - accuracy: 0.4815
Epoch 11/

In [ ]:
def sentence_generation(model, length):
    ix = [np.random.randint(vocab_size)] # 글자에 대한 랜덤 인덱스 생성
    y_char = [index_to_char[ix[-1]]] # 랜덤 익덱스로부터 글자 생성
    print(ix[-1],'번 글자',y_char[-1],'로 예측을 시작!')
    X = np.zeros((1, length, vocab_size)) # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성

    for i in range(length):
        X[0][i][ix[-1]] = 1 # X[0][i][예측한 글자의 인덱스] = 1, 즉, 예측 글자를 다음 입력 시퀀스에 추가
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [ ]:
sentence_generation(model, 100)

46 번 글자 q 로 예측을 시작!
quite strange at first; but she got used to it in a few minut i tanter i queen his proce the had-bar

'quite strange at first; but she got used to it in a few minut i tanter i queen his proce the had-bar.'

In [ ]:
sentence_generation(model, 30)

44 번 글자 o 로 예측을 시작!
out it, added the hatter. but 

'out it, added the hatter. but t'

In [ ]:
sentence_generation(model, 100)

42 번 글자 m 로 예측을 시작!
may copy it, give it away or re-use it under the terms of the project gutenberg-tm license. 1.e.6. y

'may copy it, give it away or re-use it under the terms of the project gutenberg-tm license. 1.e.6. yo'

In [ ]:
sentence_generation(model, 25)

27 번 글자 [ 로 예측을 시작!
[latem, the queens shere 

'[latem, the queens shere t'

## 글자 단위 RNN(Char RNN)으로 텍스트 생성하기

- 다 대 일(many to one) 구조의 RNN을 글자 단위로 학습시키고, 텍스트 생성하기

### 데이터에 대한 이해와 전처리

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical

In [ ]:
text='''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

In [ ]:
tokens = text.split()  # '\n 제거'
text = ' '.join(tokens)

print(text)

I get on with life as a programmer, I like to contemplate beer. But when I start to daydream, My mind turns straight to wine. Do I love wine more than beer? I like to use words about beer. But when I stop my talking, My mind turns straight to wine. I hate bugs and errors. But I just think back to wine, And I'm happy once again. I like to hang out with programming and deep learning. But when left alone, My mind turns straight to wine.


In [ ]:
char_vocab = sorted(list(set(text))) #중복을 제거한 글자 집합 생성

print(char_vocab)

[' ', "'", ',', '.', '?', 'A', 'B', 'D', 'I', 'M', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [ ]:
vocab_size = len(char_vocab)

print('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 33


In [ ]:
char_to_index = dict((c, i) for i,c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여

print(char_to_index)

{' ': 0, "'": 1, ',': 2, '.': 3, '?': 4, 'A': 5, 'B': 6, 'D': 7, 'I': 8, 'M': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15, 'g': 16, 'h': 17, 'i': 18, 'j': 19, 'k': 20, 'l': 21, 'm': 22, 'n': 23, 'o': 24, 'p': 25, 'r': 26, 's': 27, 't': 28, 'u': 29, 'v': 30, 'w': 31, 'y': 32}


- example 5개의 입력 글자 시퀀스로부터 다음 글자 시퀀스를 예측 

  - stude -> n  
  - tuden -> t  

In [ ]:
length = 11
sequences = []
for i in range(length, len(text)):
  seq = text[i-length:i] # 길이 11의 문자열을 지속적으로 만듬
  sequences.append(seq)

print('총 훈련 샘플의 수: %d' %len(sequences))

총 훈련 샘플의 수: 426


In [ ]:
print(sequences)

['I get on wi', ' get on wit', 'get on with', 'et on with ', 't on with l', ' on with li', 'on with lif', 'n with life', ' with life ', 'with life a', 'ith life as', 'th life as ', 'h life as a', ' life as a ', 'life as a p', 'ife as a pr', 'fe as a pro', 'e as a prog', ' as a progr', 'as a progra', 's a program', ' a programm', 'a programme', ' programmer', 'programmer,', 'rogrammer, ', 'ogrammer, I', 'grammer, I ', 'rammer, I l', 'ammer, I li', 'mmer, I lik', 'mer, I like', 'er, I like ', 'r, I like t', ', I like to', ' I like to ', 'I like to c', ' like to co', 'like to con', 'ike to cont', 'ke to conte', 'e to contem', ' to contemp', 'to contempl', 'o contempla', ' contemplat', 'contemplate', 'ontemplate ', 'ntemplate b', 'template be', 'emplate bee', 'mplate beer', 'plate beer.', 'late beer. ', 'ate beer. B', 'te beer. Bu', 'e beer. But', ' beer. But ', 'beer. But w', 'eer. But wh', 'er. But whe', 'r. But when', '. But when ', ' But when I', 'But when I ', 'ut when I s', 't when I

In [ ]:
x = []
for line in sequences: #전체 데이터에서 문장 샘플을 1개씩 꺼냄
  temp_x = [char_to_index[char] for char in line] # 문장 샘플에서 각 글자에 대해서 정수 인코딩 수행
  x.append(temp_x)

In [ ]:
for line in x[:5]:
  print(line)

[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18]
[0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28]
[16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17]
[14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0]
[28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]


In [ ]:
sequences = np.array(x)
xx = sequences[:, :-1]
y = sequences[:, -1]  # 맨 마지막 위치의 글자를 분리

In [ ]:
for line in x[:5]:
  print(line)

[8, 0, 16, 14, 28, 0, 24, 23, 0, 31, 18]
[0, 16, 14, 28, 0, 24, 23, 0, 31, 18, 28]
[16, 14, 28, 0, 24, 23, 0, 31, 18, 28, 17]
[14, 28, 0, 24, 23, 0, 31, 18, 28, 17, 0]
[28, 0, 24, 23, 0, 31, 18, 28, 17, 0, 21]


In [ ]:
print(y[:5])

[18 28 17  0 21]


### 원-핫 인코딩 수행

In [ ]:
sequences = [to_categorical(x, num_classes=vocab_size) for x in xx]

In [ ]:
x = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)  # y에 대한 원-핫 인코딩

In [ ]:
print(x.shape)

(426, 10, 33)


- 샘플의 수 426  
- 입력시퀀스의 길이 10  
- 각 벡터의 차원 33 

### 모델 설계

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(x.shape[1], x.shape[2])))  # (10, 33)
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])
model.fit(x, y, epochs=100, verbose=1)

Epoch 1/100
14/14 [==============================] - 1s 5ms/step - loss: 3.4462 - accuracy: 0.1573
Epoch 2/100
14/14 [==============================] - 0s 4ms/step - loss: 3.1475 - accuracy: 0.1972
Epoch 3/100
14/14 [==============================] - 0s 3ms/step - loss: 2.9878 - accuracy: 0.1972
Epoch 4/100
14/14 [==============================] - 0s 3ms/step - loss: 2.9673 - accuracy: 0.1972
Epoch 5/100
14/14 [==============================] - 0s 4ms/step - loss: 2.9328 - accuracy: 0.1972
Epoch 6/100
14/14 [==============================] - 0s 4ms/step - loss: 2.9181 - accuracy: 0.1972
Epoch 7/100
14/14 [==============================] - 0s 4ms/step - loss: 2.8865 - accuracy: 0.1972
Epoch 8/100
14/14 [==============================] - 0s 4ms/step - loss: 2.8578 - accuracy: 0.1995
Epoch 9/100
14/14 [==============================] - 0s 4ms/step - loss: 2.8176 - accuracy: 0.2019
Epoch 10/100
14/14 [==============================] - 0s 4ms/step - loss: 2.7570 - accuracy: 0.2136
Epoch 11/

In [ ]:
def sentence_generation(model, char_to_index, seq_length, seed_text, n):
  # 모델, 인덱스 정보, 문장 길이, 초기 시퀀스, 반복 횟수

  init_text = seed_text  #문장 생성에 사용할 초기 시퀀스
  sentence = ''

  for _ in range(n): # n번 반복
    encoded = [char_to_index[char] for char in seed_text]  #현재 시퀀스에 대한 정수 인코딩
    encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre')  #데이터에 대한 패딩
    encoded = to_categorical(encoded, num_classes=len(char_to_index))
    result = model.predict_classes(encoded, verbose=0)
    # 입력한 x(현재 시퀀스)에 대해서 y를 예측하고 y(예측한 글자)를 result에 저장

    for char, index in char_to_index.items():  # 만약 예측한 글자와 인덱스와 동일한 글자가 있다면
      if index == result:  # 해당 글자가 예측 글자이므로 break
        break
    
    seed_text = seed_text + char  # 현재 시퀀스 + 예측 글자를 현재 시퀀스에 변경
    sentence = sentence + char  # 예측 글자를 문장에 저장

  sentence = init_text + sentence
  return sentence

In [ ]:
print(sentence_generation(model, char_to_index, 10, 'I get on w', 100))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


I get on with life as a programmer, I like to hang out with programming and deep learning. But when I stop my 


In [ ]:
print(sentence_generation(model, char_to_index, 10, 'Do I love wine', 100))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Do I love wine more than beer? I like to hang out with programming and deep learning. But when I stop my talking, 
